## Install Requirements
******Remember to restart the kernel after installing

In [1]:
%pip install --upgrade pip
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.
Ignoring tensorflow: markers 'sys_platform == "linux"' don't match your environment
Ignoring tensorflow: markers 'sys_platform == "windows"' don't match your environment
Ignoring tensorflow-macos: markers 'sys_platform == "darwin"' don't match your environment
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os, sys, shutil
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split

## Helper Functions

In [17]:
def calc_median(frames):
    median_frame = np.median(frames, axis=0).astype(dtype=np.uint8)
    return median_frame

def doMovingAverageBGS(image, prev_frames):
    median_img = calc_median(prev_frames)
    image = cv2.absdiff(image, median_img)
    return image

In [22]:
def extractImages(pathIn, pathOut, leakRange, nonleakRange, currCountLeak, currCountNonLeak):

  '''
  Input:
    String: pathIn should be the path of the video 
    String: pathOut should be the path of the folder where data is being stored for testing or training
    Tuple: range of leak frames from video
    Tuple: range of nonleak frames from video

  Output:
    creates two subfolders in pathOut called Leaks and Nonleaks
      Leaks folder contains the frames where there are leaks
      Nonleaks folder contains the frames where there are noleaks
  '''

  leakPath = os.path.join(pathOut, "Leak")
  nonleakPath = os.path.join(pathOut, "Nonleaks")
  
  os.makedirs(leakPath, exist_ok=True)
  os.makedirs(nonleakPath, exist_ok=True)

  def helper(pathIn, pathOut, range, isLeak, currCountLeak, currCountNonLeak):
    '''
    Might need to clean this up, but this was extracted from the original extractImages from the previous implementation
    
    '''
    #setting up moving average list
    prev_imgs = []
    prev_limit = 210 #210 in paper

    start = range[0] * 1000 # converting seconds to milliseconds
    end = range[1] * 1000
    cap = cv2.VideoCapture(pathIn)
    cap.set(cv2.CAP_PROP_POS_MSEC, start)
    success = True

    if cap.isOpened():
      while success and start < end:  
          success, image = cap.read()
          image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

          start = cap.get(cv2.CAP_PROP_POS_MSEC)
          if success:

            prev_imgs.append(image)
            if len(prev_imgs) > prev_limit:
                prev_imgs.pop(0)
          
            processed_img = doMovingAverageBGS(image, prev_imgs) #to generalize might need to make this function as a parameter
            
            if isLeak:
                cv2.imwrite(os.path.join(pathOut, "leak.frame%d.jpg" % currCountLeak), processed_img)     # save frame as JPEG file
                currCountLeak += 1
            else:
                cv2.imwrite(os.path.join(pathOut, "nonleak.frame%d.jpg" % currCountNonLeak), processed_img)
                currCountNonLeak += 1
          else:
            break
      cap.release()
    cv2.destroyAllWindows()
    if isLeak:
       return currCountLeak
    else:
       return currCountNonLeak
  # call helper for both nonLeak and leak and get updated counts
  updated_currCountNonLeak = helper(pathIn, nonleakPath, nonleakRange, isLeak=False, currCountLeak=currCountLeak, currCountNonLeak=currCountNonLeak)
  updated_currCountLeak = helper(pathIn, leakPath, leakRange, isLeak=True,currCountLeak=currCountLeak, currCountNonLeak=currCountNonLeak)
  
  return updated_currCountNonLeak, updated_currCountLeak

## Setting Up Directories

In [5]:
# get generic path to directory
dir_path = os.path.dirname(os.path.realpath("__file__"))

# get all raw video data directories
data_dir = os.path.join(dir_path, 'data')

train_data_dir = os.path.join(data_dir, 'train')
test_data_dir = os.path.join(data_dir, 'test')

frame_data_dir = os.path.join(dir_path, 'frame_data_debug')
frame_train_data_dir = os.path.join(frame_data_dir, 'train')
frame_test_data_dir = os.path.join(frame_data_dir, 'test')

## Setting Up Ranges for Each Video In GasVid (Excluding 18.6m and 8.8m)

In [6]:
raw_data = np.loadtxt(os.path.join(dir_path, 'GasVid_Ranges_Seconds.csv'), skiprows=1, delimiter=',', dtype=int)

ranges = list(zip(raw_data[:, 0], raw_data[:, 1:3], raw_data[:, 3:5])) #need to upload new ranges
ranges = {ranges[i][0] : (ranges[i][1], ranges[i][2]) for i in range(len(ranges))}
len(ranges)

25

In [24]:
def read_frames_from_dir(dir_path, output_path, max_vids=None):
    cur_count = 0 
    currNonLeakCount = 0
    currLeakCount = 0
    
    for file in os.listdir(dir_path):
        if max_vids and cur_count > max_vids:
            break
        vid_path = os.path.join(dir_path, file)
        vid_id = int(os.path.basename(vid_path)[4:8])
        if vid_id not in ranges.keys():
            continue

        nonleak_start = ranges[vid_id][0][0]
        nonleak_end = ranges[vid_id][0][1]
        leak_start = ranges[vid_id][1][0]
        leak_end = ranges[vid_id][1][1]

        currNonLeakCount, currLeakCount = extractImages(vid_path, output_path, (leak_start, leak_end), (nonleak_start, nonleak_end), currLeakCount, currNonLeakCount)
        print("Video", vid_id)
        print("Current NonLeak Count", currNonLeakCount)
        print("Current Leak Count", currLeakCount)

        print('Done with', cur_count + 1, "video(s)")
        cur_count += 1
    return currNonLeakCount, currLeakCount

## Reading Frames from Data Directory and Setting Them in Frame Data Directory

In [25]:
image_dim = (240, 320)
vid_count = 1
test_count = 1

total_train_NonLeak, total_train_Leak = read_frames_from_dir(train_data_dir, frame_train_data_dir, vid_count)
print("Done with Training Data")
total_test_NonLeak, total_test_Leak = read_frames_from_dir(test_data_dir, frame_test_data_dir, test_count)
print("Done with Testing Data")
#weird bug in which it if vid_count is 1 goes on to the next one
#might need to cut in amount of samples but we will see

Video 1237
Current NonLeak Count 2387
Current Leak Count 18473
Done with 1 video(s)


KeyboardInterrupt: 

## Class Weigths

In [ ]:
nonleaks = total_train_NonLeak
leaks = total_train_Leak
total = nonleaks + leaks

weight_nonleak = (1 / nonleaks) * (total / 2.0)
weight_leak = (1 / leaks) * (total / 2.0)

class_weight = {0: weight_nonleak, 1: weight_leak}

## Setting Up Generators for Training, Validation, and Testing

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

val_split = 0.2
batch_size = 32

train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rescale=1. / 255,
    validation_split=val_split,
)

train_generator = train_datagen.flow_from_directory(
    directory=frame_train_data_dir,
    class_mode="binary",
    subset="training",
    batch_size=batch_size,
    color_mode="grayscale"

)

val_generator = train_datagen.flow_from_directory(
    directory=frame_train_data_dir,
    class_mode="binary",
    subset="validation",
    batch_size=batch_size,
    color_mode="grayscale"

)


test_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rescale=1. / 255,
)

test_generator = test_datagen.flow_from_directory(
    directory=frame_test_data_dir, 
    class_mode='binary', 
    batch_size=batch_size,
    color_mode="grayscale"

)

## Sample Weigths

In [ ]:
#TODO: calculate sample_weights using train_generator | should be similar to how predictions are made at the bottom of the notebook
#I think this should work but not sure

# A DirectoryIterator yielding tuples of (x, y) where x is a numpy array 
# containing a batch of images with shape (batch_size, *target_size, channels) 
# and y is a numpy array of corresponding labels. (In our case its train_generator)

# def sample_weights(data):
#     weights = []
#     zero_sum = 0
#     for data, _ in train_generator:
#        batch_size = data.shape[0]
#        for image_index in range(batch_size):
#             image = data[image_index]
#             summed_pixels = np.sum(image)
#             if summed_pixels == 0:
#                 weights.append(0)
#                 zero_sum += 1
#             else:
#                 # try using sqrt transformation for weight skew
#                 # weights.append(1 / np.sqrt(summed_pixels))
#                 weights.append(1 / summed_pixels)
#     train_generator.reset()
#     median_weight = np.median(weights)
#     weights = [median_weight if weight == 0 else weight for weight in weights]
#     return weights

## Model

In [ ]:
from keras import layers 
from keras import models 

model = models.Sequential() 

# Conv Pool 1
model.add(layers.Conv2D(4, (3, 3), input_shape=(240, 320, 1)))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Conv Pool 2
model.add(layers.Conv2D(8, (3, 3)))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Conv Pool 3
model.add(layers.Conv2D(8, (3, 3)))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Conv Pool4
model.add(layers.Conv2D(4, (3, 3)))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

model.add(layers.Flatten())

model.add(layers.Dense(2400, activation='relu')) 
model.add(layers.Dropout(0.5))
model.add(layers.Dense(32, activation='relu')) 
model.add(layers.Dense(1, activation='sigmoid')) 

model.summary()

## Data Visualization (Confusion Matrix and ROC Curves)

In [ ]:
#code from https://neptune.ai/blog/keras-metrics
#to use to plot confusion matrix and roc curve after each epock
#uncomment when you need it 

# import os

# from keras.callbacks import Callback
# import matplotlib.pyplot as plt
# import numpy as np
# from scikitplot.metrics import plot_confusion_matrix, plot_roc
# class PerformanceVisualizationCallback(Callback):
#     def __init__(self, model, validation_data, image_dir):
#         super().__init__()
#         self.model = model
#         self.validation_data = validation_data

#         os.makedirs(image_dir, exist_ok=True)
#         self.image_dir = image_dir

#     def on_epoch_end(self, epoch, logs={}):
#         y_pred = np.asarray(self.model.predict(self.validation_data[0]))
#         y_true = self.validation_data[1]
#         y_pred_class = np.argmax(y_pred, axis=1)

#         # plot and save confusion matrix
#         fig, ax = plt.subplots(figsize=(16,12))
#         plot_confusion_matrix(y_true, y_pred_class, ax=ax)
#         fig.savefig(os.path.join(self.image_dir, f'confusion_matrix_epoch_{epoch}'))

#        # plot and save roc curve
#         fig, ax = plt.subplots(figsize=(16,12))
#         plot_roc(y_true, y_pred, ax=ax)
#         fig.savefig(os.path.join(self.image_dir, f'roc_curve_epoch_{epoch}'))

# performance_cbk = PerformanceVisualizationCallback(
#                       model=model,
#                       validation_data=val_generator,
#                       image_dir='performance_vizualizations')

## Training Model

In [ ]:

num_epochs = 5

history = model.fit_generator(
    generator=train_generator,
    steps_per_epoch= train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps = val_generator.samples // batch_size,
    epochs = num_epochs,
    # callbacks=[performance_cbk] #uncomment once you want to use it
)

## Model Metrics Over Epochs

In [ ]:
import matplotlib.pyplot as plt 

f1 = history.history['F1Score'] 
val_f1 = history.history['val_F1Score'] 
loss = history.history['loss'] 
val_loss = history.history['val_loss'] 

epochs = range(1, len(f1) + 1) 

plt.plot(epochs, f1, 'bo', label='Training F1 Score') 
plt.plot(epochs, val_f1, 'b', label='Validation F1 Score') 
plt.title('Training and Validation F1 Score') 
plt.legend() 

plt.figure() 

plt.plot(epochs, loss, 'bo', label='Training loss') 
plt.plot(epochs, val_loss, 'b', label='Validaion loss') 
plt.title('Training loss and validation loss') 
plt.legend() 

plt.show() 

## Testing

In [ ]:
#source: https://stackoverflow.com/questions/45413712/keras-get-true-labels-y-test-from-imagedatagenerator-or-predict-generator

# Create lists for storing the predictions and labels
predictions = []
labels = []

threshold = 0.5
# Get the total number of labels in generator 
# (i.e. the length of the dataset where the generator generates batches from)
length_test = len(test_generator.labels)

# Loop over the generator
for data, label in test_generator:
    # Make predictions on data using the model. Store the results.
    preds = model.predict(data)
    processed_preds = (preds >= 0.5).flatten().astype(int)
    predictions.extend(processed_preds)

    # Store corresponding labels
    labels.extend(label)

    # We have to break out from the generator when we've processed 
    # the entire once (otherwise we would end up with duplicates). 
    if (len(label) < test_generator.batch_size) and (len(predictions) == n):
        break

In [ ]:
test_acc = np.sum(predictions == labels) / length_test
print(f'Test Accuracy is {test_acc} after training for {num_epochs} epochs on {length_test} test images')

In [ ]:
leak_indices = [i for i in range(length_test) if labels[i] == 1] #TODO: need to make sure if 1 corresponds to a leak with labels
nonleak_indices = [i for i in range(length_test) if labels[i] == 0] #TODO: need to make sure if 0 corresponds to a nonleak with labels

leak_predictions, leak_y_test = predictions[leak_indices], labels[leak_indices]
nonleak_predictions, nonleak_y_test = predictions[nonleak_indices], labels[nonleak_indices]

leak_test_acc = np.sum(leak_predictions == leak_y_test) / len(leak_y_test)
nonleak_test_acc = np.sum(nonleak_predictions == nonleak_y_test) / len(nonleak_y_test)

print(f'Leak Test accuracy is {leak_test_acc} after training for {num_epochs} epochs on {len(leak_y_test)} leak test images')
print(f'Non-Leak Test accuracy is {nonleak_test_acc} after training for {num_epochs} epochs on {len(nonleak_y_test)} non-leak test images')